# Data management for model inputs and outputs

This notebook explores the use of xarray and pydantic classes for the management of input and output surrogate model variables. For the purpose of this demo, the data for process variables has been organized into the file: `online_model/var_config.py`; however, the intention for these methods is that they will be called directly from the model build in order to organize the process variables for subsequent server use. 

Three approaches will be used in organizing the model input/outputs and their metadata:
1. Use of xarray variables
2. Pydantic class implementation with validation

Once the data has been constructed, it will be served using both the Channel Access and PVAccess servers. Here, these are converted into the appropriate dictionaries for building the process variable database items. The PVAccess server could be refactored to directly use the new data structure instead of constructing the pvdb; however, the Channel Access server uses the pvdb in its constuction so the pvdb conversion step will be necessary.

## Current data requirements:
In its current iteration, the server and client use the process variable database constructed from the model info keys provided in the hdf5 file associated with the model. This serves as a single source of truth composed at runtime for both client and server, which should be avoided going forward. The process variable database should instead be derived from an input/output collection that is saved along with the model build and may be replicated across model builds. Current data used client and server-side is outlined below:


| Attribute   | Used server-side | Used client-side | Required for pva | Required for CA |
|-------------|------------------|------------------|------------------|-----------------|
| name        | ✓                | ✓                | ✓                | ✓               |
| pv_type     | ✓                | ✓                | ✓                | ✓               |
| value *     | ✓ (inputs)       |                  | ✓ (inputs)       | ✓ (inputs)      |
| default **  |                  | ✓                |                  |                 |
| units       |                  | ✓                |                  |                 |
| value_range | ✓ (CA)           | ✓ (sliders)      |                  | ✓               |
| is_input    | ✓                | ✓                | ✓                | ✓               |
| type        | ✓ (CA)           |                  |                  | ✓               |
| precision   | ✓ (CA)           |                  |                  | ✓               |

Additionally, this collection might also be extended to encompass build instructions. This will be explored in section 3.



\* Value for outputs is not required because it is computed directly from the model before serving.
<br>
\** Defaults are currently used as placeholders. There is also a concept of a missing value default for IOCs that could be acocunted for in a similar manner.

## 1. xarray for metadata 
The process variables are each constructed using the xarray.Variable class. All variables are then used to construct a Dataset. 

In [1]:
import xarray as xr
import numpy as np
from typing import List, Tuple, Union

# This is a remnant that this work would ultimately remove
# The protocol is currently used in the assembly of the pvdb because of 
# extensions necessary for Channel Access AreaDetector variables
import  os
os.environ["PROTOCOL"] = "pva"

from online_model import DEFAULT_PRECISION

# xarray does not propogate attributes by default
xr.set_options(keep_attrs = True)

def build_variable(*, 
                   name: str, 
                   pv_type: str, 
                   value: Union[np.ndarray, float], 
                   value_range: List[float], 
                   default: Union[np.ndarray, float], 
                   dim: Tuple[str], 
                   units: str, 
                   is_input: int,
                   precision: int=DEFAULT_PRECISION) -> xr.Variable:
    
    # assign default for none value
    if value is None:
        value = default
    
    # need to convert scalar to array to work with xarray
    if isinstance(value, (float,)):
        value = np.array([value])
        
    attributes = {
              "pv_type": pv_type, 
              "range" : value_range,
              "default": default,
              "units": units,
              "name": name,
              "is_input": is_input,
              "precision": precision
             }
        
    # drop None keys, cannot write None values to hdf5 file
    # note: this could be problematic with required attributes
    #attributes = {k: v for k, v in attributes.items() if v is not None}

    variable = xr.Variable(dim,
                           value,
                           attrs=attributes
                        )
    
    return variable


# Example variable using misc data
example_variable = build_variable(
    name = "example",
    pv_type = "scalar",
    value_range=[1.000000e-01, 5.000000e-01], 
    default=3.47986980e-01,
    value= np.array([3.47986980e-01]), # MUST BE AN ARRAY!
    dim = ("length",),
    units = "mm",
    is_input = 1,
)


### Now, use the build_variable function to create xarray Dataset from our model info:

In [2]:
from online_model.var_config import VARIABLES


variables = {}

for variable, configs in VARIABLES.items():
    variables[variable] = build_variable(
        name = variable,
        pv_type = configs.get("pv_type"),
        value_range=configs.get("value_range"), 
        default=configs.get("default"),
        value= configs.get("value"),
        dim = configs.get("xarray_dim"),
        units = configs.get("units"),
        is_input = configs.get("is_input")
    )
    
    
dset = xr.Dataset(variables)
# show dset explore view
dset

<xarray.Dataset>
Dimensions:                    (C: 1, charge: 1, eV: 1, induction: 1, length: 1, m: 1, mm: 1, mm-mrad: 1, mm-rad: 1, mrad: 1, number: 1, phi: 1)
Dimensions without coordinates: C, charge, eV, induction, length, m, mm, mm-mrad, mm-rad, mrad, number, phi
Data variables:
    laser_radius               (length) float64 0.348
    maxb(2)                    (induction) float64 0.04028
    phi(1)                     (phi) float64 -7.991
    total_charge:value         (charge) float64 141.6
    in_xmin                    (length) float64 -0.000354
    in_xmax                    (length) float64 0.0003458
    in_ymin                    (length) float64 -0.0003479
    in_ymax                    (length) float64 0.0003458
    end_core_emit_95percent_x  (mm-mrad) float64 0.0
    end_core_emit_95percent_y  (mm-mrad) float64 0.0
    end_core_emit_95percent_z  (mm-rad) float64 0.0
    end_mean_kinetic_energy    (eV) float64 0.0
    end_mean_x                 (mm) float64 0.0
    end_mean_y                 (mm) float64 0.0
    end_n_particle_loss        (number) float64 0.0
    end_norm_emit_x            (mm-rad) float64 0.0
    end_norm_emit_y            (mm-rad) float64 0.0
    end_norm_emit_z            (mm-rad) float64 0.0
    end_sigma_x                (mm) float64 0.0
    end_sigma_xp               (mrad) float64 0.0
    end_sigma_y                (mm) float64 0.0
    end_sigma_yp               (mrad) float64 0.0
    end_sigma_z                (mm) float64 0.0
    end_total_charge           (C) float64 0.0
    out_xmin                   (m) float64 0.0
    out_xmax                   (m) float64 0.0
    out_ymin                   (m) float64 0.0
    out_ymax                   (m) float64 0.0
    x:y                        object None

### Prepare to serve
Note: Enforcing required attributes for our entry would require massive manual extension of the build_variable. This has the potential to result in missing required fields or incorrectly typed attributes.

In [3]:
from online_model.server import ca, pva
from online_model.model.MySurrogateModel import MySurrogateModel


def pvdb_from_xarray(dset):
    input_pvdb = {}
    output_pvdb = {}
    
    for variable in dset.keys():
        
        entry = {
            "prec": dset[variable].attrs["precision"],
            "units": dset[variable].attrs["units"],
            "range": dset[variable].attrs["range"],
            "type": "float" # For channel access
        }
        
        
        if dset[variable].attrs["is_input"]:
            
            # set values for the inputs
            if dset[variable].attrs["pv_type"] == "scalar":
                entry["value"] = dset[variable].values[0] # Have to extract our scalar value from the xarray        
                
            else:
                entry["value"] = dset[variable]
            
            input_pvdb[variable] = entry
            
        else:
            output_pvdb[variable]  = entry
    
    return input_pvdb, output_pvdb

In [6]:
# save dataset to file
import pickle

with open("online_model/files/xarray_dset.pickle",  "wb") as f:
    pickle.dump(dset, f, protocol=-1)

### Run Channel Access Server
Note: The `build_image_pvs` function used below is a utility function that adds the appropriate image process variables for the AreaDetector naming scheme. This will ultimately be included in the initial pvdb construction.

In [4]:
from online_model.util import build_image_pvs
from online_model import DEFAULT_PRECISION, DEFAULT_COLOR_MODE, IMAGE_UNITS, IMAGE_SHAPE, PREFIX, MODEL_KWARGS
from online_model.server import ca

input_pvdb, output_pvdb = pvdb_from_xarray(dset)


image_pvs = build_image_pvs(
        "x:y", 
        IMAGE_SHAPE,
        IMAGE_UNITS,
        DEFAULT_PRECISION,
        DEFAULT_COLOR_MODE,
    )

output_pvdb.update(image_pvs)

output_pvdb

server = ca.CAServer(MySurrogateModel, MODEL_KWARGS, input_pvdb, output_pvdb, PREFIX)
server.start_server()

Loaded Attributes successfully
Loaded Architecture successfully
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loaded Weights successfully
Running model...Ellapsed time: 0.06671905517578125
Initializing sim...
Running model...Ellapsed time: 0.003309011459350586
...finished initializing.
Terminating server.


### Run PVAccess server
(works with Server.ipynb)

In [ ]:
from online_model import DEFAULT_PRECISION, PREFIX, MODEL_KWARGS
from online_model.server import pva

input_pvdb, output_pvdb = pvdb_from_xarray(dset)

server = pva.PVAServer(MySurrogateModel, MODEL_KWARGS, input_pvdb, output_pvdb, PREFIX)
server.start_server()

## 2. Build variables using pydantic classes
Pydantic is a library which enforces type hinting at runtime. This means that errors in variable configurations can be caught during variable construction. 

In [7]:
from pydantic import BaseModel
from typing import Optional, List, Any, Union
from enum import Enum


# custom validator for ndarrays
class NumpyNDArray(np.ndarray):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate
        
    @classmethod
    def validate(cls, v: Any) -> np.ndarray:
        # validate data...
        if not isinstance(v, np.ndarray):
            raise TypeError('Numpy array required')
        return v

# custom validator for xarray DataArrays
class XarrayDataArray(xr.DataArray):
    __slots__ = [] # xarrray requires explicit definition of slots on subclasses
    
    @classmethod
    def __get_validators__(cls):
        yield cls.validate
        
    @classmethod
    def validate(cls, v: Any) -> xr.DataArray:
        # validate data...
        if not isinstance(v, xr.DataArray):
            raise TypeError('Xarray DataArray required')
        return v
    
class IOEnum(str, Enum):
    input = "input"
    output = "output"

class ProcessVariable(BaseModel):
    name: str
    io_type: IOEnum  #requires selection of input or output for creation
    units: Optional[str]
    # defaults for pvdb
    type: str = "float"
    precision: int = DEFAULT_PRECISION
            
    class Config:
        use_enum_values = True
        
        
class ScalarProcessVariable(ProcessVariable):
    value: Optional[float]
    default: Optional[float]
    range:  Optional[Union[NumpyNDArray, XarrayDataArray]]
    

class NDProcessVariable(ProcessVariable):
    value: Optional[NumpyNDArray]
    default: Optional[NumpyNDArray]
    range:  Optional[Union[NumpyNDArray, XarrayDataArray]]
    
# working example
test_param = NDProcessVariable(
                    name = "test", 
                    io_type = "input", 
                    value=np.array([5,  2],), 
                    default= np.array([5.0,  2.0],), 
                    range=np.array([5.0, 4.0])
            )


## Example of an incorrectly typed prcess variable:

In [8]:
test_param = ScalarProcessVariable(
                    name="test", 
                    io_type = "input", 
                    units="test", 
                    value=5.0, 
                    default= np.ndarray([5,  2],), # for a scalar parameter, a float should be passed
                    range=np.ndarray([5, 4])
                )

ValidationError: 1 validation error for ScalarProcessVariable
default
  value is not a valid float (type=type_error.float)

## Similar to the xarray config, we can use classes to set up process variables on a case by case basis

In [10]:
variables = []

for variable, configs in VARIABLES.items():
    
    if configs["is_input"] == 1:
        io_type = "input"
        
    else:
        io_type = "output"
    
    
    if configs["pv_type"] == "scalar":
        var = ScalarProcessVariable(
            name = variable,
            io_type = io_type,
            **configs
        )
    
    elif configs["pv_type"] == "image":
        var =  NDProcessVariable(
            name = variable,
            io_type = io_type,
            **configs
        )
    
    variables.append(var)

## Create pvdb from the class instances

In [11]:
def pvdb_from_classes(variables):
    input_pvdb = {}
    output_pvdb = {}
    
    for variable in variables:
        # no manual formatting needed and have control over what is included/excluded
        entry = variable.dict(exclude_unset = True, exclude={"io_type",})
        
        if variable.io_type == "input":
            input_pvdb[variable.name] = entry
            
        elif variable.io_type == "output":
            output_pvdb[variable.name]  = entry
            
        else:
            # pydantic enum validation will prohibit any other assignment
            pass 
    
    return input_pvdb, output_pvdb

In [13]:
# save vars to file
import pickle

with open("online_model/files/pydantic_variables.pickle",  "wb") as f:
    pickle.dump(variables, f)

### Run Channel Access Server
Note: The `build_image_pvs` function used below is a utility function that adds the appropriate image process variables for the AreaDetector naming scheme. This will ultimately be included in the initial pvdb construction.

In [ ]:
from online_model.util import build_image_pvs
from online_model import DEFAULT_PRECISION, DEFAULT_COLOR_MODE, IMAGE_UNITS, IMAGE_SHAPE, PREFIX, MODEL_KWARGS
from online_model.server import ca

input_pvdb, output_pvdb = pvdb_from_classes(variables)

print(output_pvdb)

image_pvs = build_image_pvs(
        "x:y", 
        IMAGE_SHAPE,
        IMAGE_UNITS,
        DEFAULT_PRECISION,
        DEFAULT_COLOR_MODE,
    )

output_pvdb.update(image_pvs)

server = ca.CAServer(MySurrogateModel, MODEL_KWARGS, input_pvdb, output_pvdb, PREFIX)
server.start_server()

### Run PVAccess Server 
(works with Server.ipynb)

In [ ]:
from online_model import DEFAULT_PRECISION, PREFIX, MODEL_KWARGS
from online_model.server import pva

input_pvdb, output_pvdb = pvdb_from_classes(variables)

server = pva.PVAServer(MySurrogateModel, MODEL_KWARGS, input_pvdb, output_pvdb, PREFIX)
server.start_server()


# Conclusions

There are a couple of strange things about the xarray implementation.
- The array dimension field is unused in our server/client famework; however, xarray variables require dimension definition. This will contradict the openPMD standard for dimensionless quantities. 
- A value field must be passed to construct the array. This isn't consistent with how output variables are used currently as they don't require values for initial construction.
- Have to pass scalar process variables as an array and then sample that array when extracting value
- In order to access the indexing features of xarray, a strict pythonic naming scheme has to be enforced for the input/output names (For example, data_set.laser_radius.value, vs, data_set\["phi(1)"\].value). Use of pythonic aliases for the variables within the xarray structure could be augmented by an attribute with the proper name; however, this is redundant and probably would lead to confusion.
- Addition of new attribute = more function arguments


## Beyond server config
Other configurations will be needed in order to expand this framework beyond the current model implementation. On the client side, instructions for rendering must be passed on the basis of process variable type. In order to do this, the type of the variable must be known. With xarray this requires the introspection of attributes for each variable. With the class implementation, this could be enforced with a type validation on the process variable class. 

Client-side configurations will also require the abstraction of indicators that are currently hard-coded. For example, slider inclusion/exclusion will need to be abstracted before accomodating other models. This could be accomplished by an exposure attribute or by the optional inclusion of SliderConfig settings, that could enforce additional range constraints, set the step size, etc.. Other controls may be required beyond sliders, each with their own, specific and sometimes optional settings. Pydantic models are well suited to this application and may be favorable over attribute expansion in xarray.

# Other considerations

- How will these inputs and outputs be saved? Is it a requirment for them to be nested in the current hdf5 or, can they be pickled in their own designated file?
